# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

%matplotlib inline

print(tf.__version__)


# Carregamento de dados

In [ ]:
TRAINING_DATASET_SOURCE = 'datasets/training_data.csv'
TEST_DATASET_SOURCE = 'datasets/test_data.csv'

train_df = pd.read_csv(TRAINING_DATASET_SOURCE)
test_df = pd.read_csv(TEST_DATASET_SOURCE)

# SEED utilizada

In [ ]:
SEED = 101

# Preparação dos dados

In [ ]:
categorical_to_numerical = {
    'avg_rain': {
        'Sem Chuva': 0,
        'chuva fraca': 1,
        'chuva moderada': 2,
        'chuva forte': 3
    },
    'luminosity': {
        'LOW_LIGHT': 0,
        'LIGHT': 1,
        'DARK': 2,
    }
}

incidents_to_numerical = {
    'incidents': {
        'None': 0,
        'Low': 1,
        'Medium': 2,
        'High': 3,
        'Very_High': 4,
    }
}

In [ ]:
def neural_network_data_preparation(df: pd.DataFrame) -> pd.DataFrame:
    dropped_columns = ['city_name', 'magnitude_of_delay', 'avg_precipitation']

    prep_df = df.drop(dropped_columns, axis=1)

    ### Extrair a hora e dia da semana da feature 'record_date'
    record_date = pd.DatetimeIndex(prep_df['record_date'])

    prep_df['record_date_hour'] = record_date.hour
    prep_df['record_date_day'] = record_date.day
    prep_df['record_date_month'] = record_date.month
    prep_df['record_date_weekday'] = record_date.weekday

    prep_df.drop(columns=['record_date'], inplace=True)

    ### Quantificar a feature 'affected_roads' para o número único de estradas afetadas
    road_quantity = []
    for line in prep_df['affected_roads']:
        res = set(str(line).split(','))
        res2 = [elem for elem in res if elem != '']
        count = len(res2)
        road_quantity.append(count)

    prep_df['affected_roads'] = road_quantity

    prep_df.replace(categorical_to_numerical, inplace=True)

    ### Target
    if 'incidents' in prep_df.columns:
        prep_df.replace(incidents_to_numerical, inplace=True)

    return prep_df

In [ ]:
X = neural_network_data_preparation(train_df)
y = X['incidents']

X.drop(columns=['incidents'], inplace=True)

In [ ]:
scaler_X = MinMaxScaler(feature_range=(0, 1)).fit(X)
X_scaled = pd.DataFrame(scaler_X.transform(X[X.columns]), columns=X.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=SEED)

# Construção da estrutura da rede neuronal

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt


def build_model(hp):
    # Create a sequential model (with three layers - last one is the output)
    model = keras.Sequential()

    input_hp_units = hp.Int('units', min_value=5, max_value=200, step=5)

    model.add(Dense(input_hp_units, input_dim=12, activation='relu'))

    hp_units = hp.Int('units', min_value=5, max_value=200, step=5)

    hp_activation = hp.Choice("activation", ["relu", "tanh", 'sigmoid'])

    model.add(Dense(hp_units, activation=hp_activation))

    # Output Layer
    model.add(Dense(5, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5])

    # Model compilation
    model.compile(loss=SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  metrics=['accuracy'])

    return model


In [ ]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="trials",
    project_name="competition",
)


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X, y, epochs=500, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X, y, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X, y, epochs=best_epoch, validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(X, y)
print("[test loss, test accuracy]:", eval_result)

-------
-------
-------

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=32)

In [ ]:
history_dict = history.history

acc = history_dict['accuracy']
loss = history_dict['loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, acc, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
predicts = model.predict(X_test)

categories_predicted = [np.argmax(pred) for pred in predicts]

categories_predicted

In [ ]:
y_test

# Obter as previsões no dataset de submissão

In [ ]:
X = neural_network_data_preparation(test_df)

scaler_X = MinMaxScaler(feature_range=(0, 1)).fit(X)
X_scaled = pd.DataFrame(scaler_X.transform(X[X.columns]), columns=X.columns)

X_scaled

In [ ]:
categories_prob_predictions = model.predict(X_scaled)

categories_prob_predictions

In [ ]:
numerical_predictions = [np.argmax(pred) for pred in categories_prob_predictions]

numerical_predictions[:10]

In [ ]:
numerical_predictions_df = pd.DataFrame(numerical_predictions)

incidents_categories = {
    0: 'None',
    1: 'Low',
    2: 'Medium',
    3: 'High',
    4: 'Very_High',
}

predictions_df = numerical_predictions_df.replace(incidents_categories)

In [ ]:
predictions_df.index += 1

predictions_df.to_csv("submission.csv", header=['Incidents'], index_label='RowId')